In [1]:
import os
from os import walk
import numpy as np
import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy.linalg as linalg

from PIL import Image

In [2]:
'''
Implement the functions that were not implemented and complete the
parts of main according to the instructions in comments.
'''

def reconstructed_image(D,c,num_coeffs,X_mean,n_blocks,im_num):
    '''
    This function reconstructs an image X_recon_img given the number of
    coefficients for each image specified by num_coeffs
    '''
    
    '''
        Parameters
    ---------------
    c: np.ndarray
        a n x m matrix  representing the coefficients of all the image blocks.
        n represents the maximum dimension of the PCA space.
        m is (number of images x n_blocks**2)

    D: np.ndarray
        an N x n matrix representing the basis vectors of the PCA space
        N is the dimension of the original space (number of pixels in a block)

    im_num: Integer
        index of the image to visualize

    X_mean: np.ndarray
        a matrix representing the mean block.

    num_coeffs: Integer
        an integer that specifies the number of top components to be
        considered while reconstructing
        

    n_blocks: Integer
        number of blocks comprising the image in each direction.
        For example, for a 256x256 image divided into 64x64 blocks, n_blocks will be 4
    '''
    
    c_im = c[:num_coeffs,n_blocks*n_blocks*im_num:n_blocks*n_blocks*(im_num+1)]
    D_im = D[:,:num_coeffs]
    
    
    #TODO: Enter code below for reconstructing the image X_recon_img
    #......................
    #......................
    #X_recon_img = ........
    X_recon_img = np.zeros((256,256))
    temp = np.dot(c_im.T,D_im.T)
    sz = X_mean.shape[0]
    for i in range(n_blocks):
        for j in range(n_blocks):
            X_recon_img[i*sz:(i+1)*sz,j*sz:(j+1)*sz] = temp[i*n_blocks+j].reshape(sz,sz) + X_mean
    return X_recon_img

In [3]:
def plot_reconstructions(D,c,num_coeff_array,X_mean,n_blocks,im_num):
    '''
    Plots 9 reconstructions of a particular image using D as the basis matrix and coeffiecient
    vectors from c

    Parameters
    ------------------------
        num_coeff_array: Iterable
            an iterable with 9 elements representing the number of coefficients
            to use for reconstruction for each of the 9 plots
        
        c: np.ndarray
            a n x m matrix  representing the coefficients of all the image blocks.
            n represents the maximum dimension of the PCA space.
            m is (number of images x n_blocks**2)

        D: np.ndarray
            an N x l matrix representing l basis vectors of the PCA space
            N is the dimension of the original space (number of pixels in a block)

        n_blocks: Integer
            number of blocks comprising the image in each direction.
            For example, for a 256x256 image divided into 64x64 blocks, n_blocks will be 4

        X_mean: basis vectors represent the divergence from the mean so this
            matrix should be added to all reconstructed blocks

        im_num: Integer
            index of the image to visualize
    '''
    f, axarr = plt.subplots(3,3)
    for i in range(3):
        for j in range(3):
            plt.axes(axarr[i,j])
            plt.imshow(reconstructed_image(D,c,num_coeff_array[i*3+j],X_mean,n_blocks,im_num))
            
    f.savefig('output/hw1a_{0}_im{1}.png'.format(n_blocks, im_num))
    plt.close(f)

In [4]:
def plot_top_16(D, sz, imname):
    '''
    Plots the top 16 components from the basis matrix D.
    Each basis vector represents an image block of shape (sz, sz)

    Parameters
    -------------
    D: np.ndarray
        N x n matrix representing the basis vectors of the PCA space
        N is the dimension of the original space (number of pixels in a block)
        n represents the maximum dimension of the PCA space (assumed to be atleast 16)

    sz: Integer
        The height and width of each block

    imname: string
        name of file where image will be saved.
    '''
    #TODO: Obtain top 16 components of D and plot them
    f, axarr = plt.subplots(4,4)
    for i in range(16):
        plt.axes(axarr[int(np.floor(i/4)),np.mod(i,4)])
        plt.imshow(D.T[i].reshape(sz,sz))
            
    f.savefig(imname)
    plt.close(f)

In [7]:
def main():
    '''
    Read here all images(grayscale) from Fei_256 folder
    into an numpy array Ims with size (no_images, height, width).
    Make sure the images are read after sorting the filenames
    '''
    import datetime
    print (datetime.datetime.now())
    for path, dirs, files in os.walk('Fei_256'):
        l = len(files)
    Ims = np.zeros((l,256,256))
    for i,name in zip(range(l),files):
        Ims[i] = np.array(Image.open('Fei_256/'+ name), dtype= float)
    #TODO: Read all images into a numpy array of size (no_images, height, width)
    
    szs = [8, 32, 64]
    num_coeffs = [range(1, 10, 1), range(3, 30, 3), range(5, 50, 5)]
    
    for sz, nc in zip(szs, num_coeffs):
        '''
        Divide here each image into non-overlapping blocks of shape (sz, sz).
        Flatten each block and arrange all the blocks in a
        (no_images*n_blocks_in_image) x (sz*sz) matrix called X
        ''' 
        n_blocks = int(256/sz)
        X = np.zeros((l*n_blocks*n_blocks,sz*sz))
        for i in range(l):
            for j in range(n_blocks):
                for k in range(n_blocks):
                    X[i*n_blocks*n_blocks+j*n_blocks+k,:] = Ims[i][j*sz:(j+1)*sz,k*sz:(k+1)*sz].flatten()
        #TODO: Write a code snippet that performs as indicated in the above comment
        
        X_mean = np.mean(X, 0) #mean of each column
        X = X - np.repeat(X_mean.reshape(1, -1), X.shape[0], 0)

        '''
        Perform eigendecomposition on X^T X and arrange the eigenvectors
        in decreasing order of eigenvalues into a matrix D
        '''
        la,D = np.linalg.eigh(np.dot(X.T,X)) #la means lambda
        ind = (-la).argsort()
        D = D[:,ind]
        
        #TODO: Write a code snippet that performs as indicated in the above comment
        
        c = np.dot(D.T, X.T)
        
        for i in range(0, 200, 10):
            plot_reconstructions(D=D, c=c, num_coeff_array=nc, X_mean=X_mean.reshape((sz, sz)), n_blocks=int(256/sz), im_num=i)

        plot_top_16(D, sz, imname='output/hw1a_top16_{0}.png'.format(sz))
    print (datetime.datetime.now())

if __name__ == '__main__':
    main()
    
    

2016-10-04 16:50:41.135003
2016-10-04 16:52:21.261357
